In [1]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
import nltk
nltk.download("stopwords")
import en_core_web_sm
import spacy
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords

nlp = spacy.load("en_core_web_lg", disable=['ner', 'parser', "tok2vec"])
stops = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()
behold = pd.read_excel(io.BytesIO(uploaded['Behold+product+data+04262021.xlsx']))

Saving Behold+product+data+04262021.xlsx to Behold+product+data+04262021 (1).xlsx


In [55]:
behold["description"] = behold["description"].str.replace("\n", "")
behold.head()

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active,cleaned_description
0,01EX0PN4J9WRNZH5F93YEX6QAF,Others,Others,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirtavailable in black an...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True,-PRON- signature khadi shirt available black w...
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Others,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True,mid - length dress ruffle adjustable strap . b...
2,01EY4Y1BW8VZW51BWG5VZY82XW,Others,Others,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False,ibi slip raw red knit sneaker woman
3,01EY50E27A0P5V6KCW01XPDB43,Others,Others,IBI Slip On Black Knit Sneaker Women,NaN,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False,ibi slip black knit sneaker woman
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Others,Others,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,nan,NaN,CATIBA PRO Skate Black Suede and Canvas Contra...,False,nan


In [9]:
behold['description'] = behold['description'].astype(str)
behold.dtypes

product_id             object
brand                  object
brand_category         object
name                   object
details                object
created_at             object
brand_canonical_url    object
description            object
brand_description      object
brand_name             object
product_active           bool
dtype: object

In [10]:
def cleaner(text, lowercase, remove_stopwords):
    if lowercase:
        text = text.lower()
    text = nlp(text)
    lemmatized = list()
    for word in text:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

behold['cleaned_description'] = behold['description'].apply(cleaner, lowercase=True, remove_stopwords=True)

In [51]:
behold

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active,cleaned_description
0,01EX0PN4J9WRNZH5F93YEX6QAF,Others,Others,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True,-PRON- signature khadi shirt available black w...
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Others,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True,mid - length dress ruffle adjustable strap . b...
2,01EY4Y1BW8VZW51BWG5VZY82XW,Others,Others,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False,ibi slip raw red knit sneaker woman
3,01EY50E27A0P5V6KCW01XPDB43,Others,Others,IBI Slip On Black Knit Sneaker Women,NaN,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False,ibi slip black knit sneaker woman
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Others,Others,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,nan,NaN,CATIBA PRO Skate Black Suede and Canvas Contra...,False,nan
...,...,...,...,...,...,...,...,...,...,...,...,...
61350,01EYB5ERGYPFNGM6C9QK7Q9EV0,Others,Others,BowVida Mule in Black Suede Kidskin,"Feminine flat mules.\nSquare shape, V line on ...",2021-02-12 13:04:05.149 UTC,https://www.nomasei.com/en/produit/bowvida-mul...,The flat BowVida mule in black suede is the id...,The flat BowVida mule in black suede is the id...,BowVida Mule in Black Suede Kidskin,False,flat bowvida mule black suede ideal spring sum...
61351,01EHWTBFP368Q035FW95TRJDAY,Others,Others,Sandale Vida Mule In Tangerine Suede Kidskin,"Feminine flat mules.\nSquare shape, V line on ...",2020-09-10 20:12:19.514 UTC,https://www.nomasei.com/en/produit/vida-mule-i...,The flat Vida mule in tangerine suede is the c...,The flat Vida mule in tangerine suede is the c...,Sandale Vida Mule In Tangerine Suede Kidskin,False,flat vida mule tangerine suede comfortable fem...
61352,01EHWTCFTPPSCW10D4XBQZF28H,Others,Others,BowVida Mule in Fuschia Suede Kidskin,"Feminine flat mules.\nSquare shape, V line on ...",2020-09-10 20:12:52.43 UTC,https://www.nomasei.com/en/produit/mule-bowvid...,The flat BowVida mule in fuschia suede is the ...,The flat BowVida mule in fuschia suede is the ...,BowVida Mule in Fuschia Suede Kidskin,False,flat bowvida mule fuschia suede ideal spring s...
61353,01EYB5B5FH7JESXF82ZEMVXZMS,Others,Others,Vida Mule in Silver Metalized Leather,"Feminine flat mules.\nSquare shape, V line on ...",2021-02-12 13:02:07.344 UTC,https://www.nomasei.com/en/produit/vida-mule-i...,The flat Vida mule in silver metalized leather...,The flat Vida mule in silver metalized leather...,Vida Mule in Silver Metalized Leather,False,flat vida mule silver metalize leather comfort...


In [53]:
behold.iloc[0,11]

'-PRON- signature khadi shirt available black white easy wear beach city . -PRON- promise top -PRON- go - warm weather item . perfect blazer . hand loom weave stripe khadi cotton . slightly sheer get soft every wash . ship first week april color : white black — length 27 " width 26.5 " — one size fit — grid khadi cotton'

In [54]:
behold.iloc[0,7]

'Our signature khadi shirt\navailable in black and white\nEasy to wear from beach to city. We promise this top will be your go-to warm\nweather item. Perfect under a blazer. Hand loomed woven stripe in khadi cotton.\n Slightly sheer and gets softer with every wash. Ships First week of April\nCOLOR: WHITE OR BLACK\n—  Length 27"  and  Width  26.5" —  One size fits most —  Grid khadi cotton'

In [22]:
brands = pd.Series(behold['brand'].value_counts())
brands

7 For All Mankind      9011
Rails                  2864
Intentionally Blank    2534
A.L.C.                 2092
Rachel Comey           2081
                       ... 
WILLOW                    1
CASASOLA                  1
Mother of Pearl           1
Alanui                    1
Dodo Bar Or               1
Name: brand, Length: 386, dtype: int64

In [29]:
top30 = list(brands[0:30].index)
len(top30)

30

In [39]:
for i in range(len(behold)):
  if behold.iloc[i,1] not in top30:
    behold.iloc[i,1] = 'Others'

In [40]:
behold['brand'].value_counts()

Others                   15589
7 For All Mankind         9011
Rails                     2864
Intentionally Blank       2534
A.L.C.                    2092
Rachel Comey              2081
Misa                      2030
Studio 189                1956
ASTR the Label            1942
lemlem                    1821
Simon Miller              1451
Cynthia Rowley            1347
Outerknown                1338
Chufy                     1209
Faherty                   1204
M.M.LaFleur               1192
Janessa Leone             1119
Araks                     1081
Sea                       1053
BROCHU WALKER             1001
Tanya Taylor               991
Clare V.                   922
Nili Lotan                 717
Les Girls Les Boys         695
Prism                      667
Sandy Liang                663
6397                       652
Ancient Greek Sandals      618
Alo Yoga                   525
Collina Strada             501
Whit                       489
Name: brand, dtype: int64

In [44]:
behold['brand'] = behold['brand'].astype(str)

In [56]:
X, y = behold.iloc[:, 11], behold.iloc[:, 1]
X

0        -PRON- signature khadi shirt available black w...
1        mid - length dress ruffle adjustable strap . b...
2                      ibi slip raw red knit sneaker woman
3                        ibi slip black knit sneaker woman
4                                                      nan
                               ...                        
61350    flat bowvida mule black suede ideal spring sum...
61351    flat vida mule tangerine suede comfortable fem...
61352    flat bowvida mule fuschia suede ideal spring s...
61353    flat vida mule silver metalize leather comfort...
61354    flat bowvida mule bronze leather ideal spring ...
Name: cleaned_description, Length: 61355, dtype: object

In [57]:
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer

class Text2TfIdfTransformer(BaseEstimator):

    def __init__(self):
        self._model = TfidfVectorizer()
        pass

    def fit(self, df_x, df_y=None):
        self._model.fit(df_x)
        return self

    def transform(self, df_x):
        return self._model.transform(df_x)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
                     test_size=0.2,
                     random_state = 0,
                     stratify=y)

In [62]:
from sklearn.pipeline import Pipeline
import xgboost as xgb
pl_xgb_tf_idf = Pipeline(steps=[('tfidf',Text2TfIdfTransformer()),
                         ('xgboost', xgb.XGBClassifier(objective='multi:softmax'))])
model = pl_xgb_tf_idf.fit(X_train, y_train)
model.score(X_test, y_test)

0.8370955912313585